[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostafaelaraby/wavegan-pytorch/blob/master/notebook.ipynb)


## Cloning Repo and setup of dependencies

In [0]:
!git clone https://github.com/mostafaelaraby/wavegan-pytorch.git

Cloning into 'wavegan-pytorch'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 17), reused 29 (delta 8), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [0]:
import os
os.chdir('wavegan-pytorch')

In [0]:
!pip3  install -r requirements.txt

## Downloading Data

### Speech Commands Zero through Nine (SC09)

In [0]:
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz

--2020-03-17 17:09:56--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541227122 (516M) [application/octet-stream]
Saving to: ‘sc09.tar.gz’

sc09.tar.gz         100%[===================>] 516.15M  6.19MB/s    in 89s     

2020-03-17 17:11:26 (5.77 MB/s) - ‘sc09.tar.gz’ saved [541227122/541227122]



In [0]:
!tar -xvf sc09.tar.gz

Streaming output truncated to the last 5000 lines.
sc09/train/Eight_b4aa9fef_nohash_3.wav
sc09/train/Eight_b4aa9fef_nohash_4.wav
sc09/train/Eight_b4bef564_nohash_0.wav
sc09/train/Eight_b52bd596_nohash_0.wav
sc09/train/Eight_b544d4fd_nohash_0.wav
sc09/train/Eight_b5552931_nohash_0.wav
sc09/train/Eight_b5552931_nohash_1.wav
sc09/train/Eight_b3bb4dd6_nohash_0.wav
sc09/train/Eight_b5552931_nohash_2.wav
sc09/train/Eight_b71ebf79_nohash_0.wav
sc09/train/Eight_b8872c20_nohash_0.wav
sc09/train/Eight_b9f46737_nohash_1.wav
sc09/train/Eight_baeac2ba_nohash_1.wav
sc09/train/Eight_bd061bef_nohash_0.wav
sc09/train/Eight_be91a00a_nohash_0.wav
sc09/train/Eight_c0445658_nohash_2.wav
sc09/train/Eight_c120e80e_nohash_5.wav
sc09/train/Eight_c1e0e8e3_nohash_1.wav
sc09/train/Eight_c4500713_nohash_1.wav
sc09/train/Eight_c71e3acc_nohash_1.wav
sc09/train/Eight_b5552931_nohash_3.wav
sc09/train/Eight_b5552931_nohash_4.wav
sc09/train/Eight_b575b5fb_nohash_0.wav
sc09/train/Eight_b575b5fb_nohash_1.wav
sc09/train/Ei

### Drum sound effects

In [0]:
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/drums.tar.gz

--2020-03-17 17:11:38--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/drums.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35264807 (34M) [application/octet-stream]
Saving to: ‘drums.tar.gz’

drums.tar.gz        100%[===================>]  33.63M  5.67MB/s    in 6.0s    

2020-03-17 17:11:44 (5.64 MB/s) - ‘drums.tar.gz’ saved [35264807/35264807]



In [0]:
!tar -xvf drums.tar.gz

drums/
drums/train/
drums/train/Kick_00003.wav
drums/train/Snare_00844.wav
drums/train/Hat_01536.wav
drums/train/Kick_00239.wav
drums/train/Tom_02275.wav
drums/train/Snare_00013.wav
drums/train/Kick_01191.wav
drums/train/Snare_00631.wav
drums/train/Kick_01901.wav
drums/train/Tom_02334.wav
drums/train/Hat_00539.wav
drums/train/Kick_01007.wav
drums/train/Tom_00471.wav
drums/train/Clap_00651.wav
drums/train/Tom_01468.wav
drums/train/Snare_00595.wav
drums/train/Tom_00189.wav
drums/train/Ride_01233.wav
drums/train/Kick_01621.wav
drums/train/Snare_01158.wav
drums/train/Snare_00943.wav
drums/train/Crash_00564.wav
drums/train/Snare_01266.wav
drums/train/Hat_01670.wav
drums/train/Hat_00691.wav
drums/train/Tom_01797.wav
drums/train/Cowbell_00138.wav
drums/train/Hat_00571.wav
drums/train/Tom_01469.wav
drums/train/Kick_01491.wav
drums/train/Hat_00149.wav
drums/train/Snare_02170.wav
drums/train/Kick_02144.wav
drums/train/Tom_00882.wav
drums/train/Snare_01255.wav
drums/train/Snare_00697.wav
drums/tr

### Bach piano performances

In [0]:
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/mancini_piano.tar.gz

--2020-03-17 17:11:46--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/mancini_piano.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433417623 (413M) [application/octet-stream]
Saving to: ‘mancini_piano.tar.gz’

mancini_piano.tar.g 100%[===================>] 413.34M  8.79MB/s    in 81s     

2020-03-17 17:13:07 (5.12 MB/s) - ‘mancini_piano.tar.gz’ saved [433417623/433417623]



In [0]:
!tar -xvf mancini_piano.tar.gz

piano/test/
piano/test/17.wav
piano/test/18.wav
piano/train/
piano/train/00.wav
piano/train/01.wav
piano/train/02.wav
piano/train/03.wav
piano/train/04.wav
piano/train/05.wav
piano/train/06.wav
piano/train/07.wav
piano/train/08.wav
piano/train/09.wav
piano/train/10.wav
piano/train/11.wav
piano/train/12.wav
piano/train/13.wav
piano/train/14.wav
piano/valid/
piano/valid/15.wav
piano/valid/16.wav


### Params

In [0]:
%cat params.py
# copy cell content to next cell and edit your params then run 

In [49]:
%%writefile params.py
import torch
import random
import numpy as np
import logging
import os

#############################
# DataSet Path
#############################s
target_signals_dir = 'piano'
#############################
# Model Params
#############################
model_prefix = 'test' # name of the model to be saved
n_iterations = 100000
lr_g = 1e-4
lr_d = 3e-4 # you can use with discriminator having a larger learning rate than generator instead of using n_critic updates ttur https://arxiv.org/abs/1706.08500
beta1 = 0.5
beta2 = 0.999
decay_lr = False # used to linearly deay learning rate untill reaching 0 at iteration 100,000
generator_batch_size_factor = 1 # in some cases we might try to update the generator with double batch size used in the discriminator https://arxiv.org/abs/1706.08500
n_critic = 1 # update generator every n_critic steps if lr_g = lr_d the n_critic's default value is 5 
# gradient penalty regularization factor.
p_coeff = 10
batch_size = 10
noise_latent_dim = 100  # size of the sampling noise
model_capacity_size = 32    # model capacity during training can be reduced to 32 for larger window length of 2 seconds and 4 seconds
# rate of storing validation and costs params
store_cost_every = 300
progress_bar_step_iter_size = 400
#############################
# Backup Params
#############################
take_backup = True
backup_every_n_iters = 1000
save_samples_every = 1000
output_dir = 'output'
if not(os.path.isdir(output_dir)):
    os.makedirs(output_dir)
#############################
# Audio Reading Params
#############################
window_length = 32768 #[16384, 32768, 65536] in case of a longer window change model_capacity_size to 32
sampling_rate = 16000
normalize_audio = True 
num_channels = 1

#############################
# Logger init
#############################
LOGGER = logging.getLogger('wavegan')
LOGGER.setLevel(logging.DEBUG)
#############################
# Torch Init and seed setting
#############################
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# update the seed
manual_seed = 2019 
random.seed(manual_seed)
torch.manual_seed(manual_seed)
np.random.seed(manual_seed)
if cuda:
    torch.cuda.manual_seed(manual_seed)
    torch.cuda.empty_cache()


Overwriting params.py


## Model Training

In [0]:
!python3  train.py

  0% 1/250 [00:31<2:12:36, 31.95s/it, Loss_D WD=-0.020448340103030205, Loss_G=-1.3514044284820557, Val_G=-0.03283483907580376]<Figure size 1800x5000 with 2 Axes>
